In [4]:
help(solver)

Help on Solver in module ortools.linear_solver.pywraplp object:

class Solver(builtins.object)
 |  Methods defined here:
 |  
 |  Add(self, constraint, name='')
 |  
 |  BoolVar(self, name:'std::string const &') -> 'operations_research::MPVariable *'
 |  
 |  Clear(self) -> 'void'
 |  
 |  ComputeConstraintActivities(self) -> 'std::vector< double,std::allocator< double > >'
 |  
 |  ComputeExactConditionNumber(self) -> 'double'
 |  
 |  Constraint(self, *args) -> 'operations_research::MPConstraint *'
 |  
 |  EnableOutput(self) -> 'void'
 |  
 |  ExportModelAsLpFormat(self, obfuscated:'bool') -> 'std::string'
 |  
 |  ExportModelAsMpsFormat(self, fixed_format:'bool', obfuscated:'bool') -> 'std::string'
 |  
 |  IntVar(self, lb:'double', ub:'double', name:'std::string const &') -> 'operations_research::MPVariable *'
 |  
 |  InterruptSolve(self) -> 'bool'
 |  
 |  Iterations(self) -> 'int64'
 |  
 |  LoadModelFromProto(self, input_model:'operations_research::MPModelProto const &') -> 's

In [9]:
# from ortools.linear_solver import pywraplp as mip
# import numpy.linalg as la
# import numpy as np
# import networkx as nx

# # unaries
# A1 = 1
# A2 = 1
# A3 = 1

# B1 = 1
# B2 = 1
# B3 = 1

# C1 = 1
# C2 = 1
# C3 = 1

# V = [A1, A2, A3, B1, B2, B3, C1, C2, C3]

# E_c = [(0, 3, 6), (4, 7)]
# E_v = [(0, 1, 2), (3, 4, 5), (6, 7, 8)]

# # --------------------------------
# solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Sum = []
# Rho = {}
# #Chi = {}
# n = len(V)
# for idx in range(n):
#     Rho[idx] = solver.BoolVar('rho[%i]' % idx)

# s = solver.Sum(
#     Rho[idx] * V[idx] for idx in range(n))
# Sum.append(s)

# for clique in E_c:
#     solver.Add(solver.Sum(Rho[node] for node in clique) <= 1)

# for clique in E_v:
#     s = solver.Sum(min(Rho[a], Rho[b]) for a in clique for b in clique if a < b)
#     Sum.append(s)

# # for clique in E_v:
# #     for a in clique:
# #         for b in clique:
# #             if a >= b:
# #                 continue
# #             Chi[a, b] = solver.BoolVar('chi[%i,%i]' % (a, b))
    
# #     s = solver.Sum(Chi[a, b] * 1 for a in clique for b in clique if b > a)
# #     Sum.append(s)
    
# #     for a in clique:
# #         for b in clique:
# #             if a >= b:
# #                 continue
# #             solver.Add(2 * Chi[a, b] <= Rho[a] + Rho[b])
    
# solver.Maximize(solver.Sum(Sum))
# RESULT = solver.Solve()
# print("Time = ", solver.WallTime(), " ms")
# print("result:", RESULT)
# print('\nTotal cost:', solver.Objective().Value())

# for idx in range(n):
#     print('node ' + str(idx) + ' -> ', Rho[idx].solution_value())

# # print('\n===========================')
# # for (a,b), v in Chi.items():
# #     print(str((a, b)) + ' -> ', v.solution_value())

ValueError: Operators "<" and ">" not supported with the linear solver

In [ ]:
# from ortools.linear_solver import pywraplp as mip
# import numpy.linalg as la
# import numpy as np
# import networkx as nx

# # unaries
# A1 = 1
# A2 = 1
# A3 = 1

# B1 = 1
# B2 = 1
# B3 = 1

# C1 = 1
# C2 = 1
# C3 = 1

# V = [A1, A2, A3, B1, B2, B3, C1, C2, C3]

# E_c = [(0, 3, 6), (4, 7)]
# E_v = [(0, 1, 2), (3, 4, 5), (6, 7, 8)]

# # --------------------------------
# solver = mip.Solver('t', mip.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# Sum = []
# Rho = {}
# Chi = {}
# n = len(V)
# for idx in range(n):
#     Rho[idx] = solver.BoolVar('rho[%i]' % idx)

# s = solver.Sum(
#     Rho[idx] * V[idx] for idx in range(n))
# Sum.append(s)

# for clique in E_c:
#     solver.Add(solver.Sum(Rho[node] for node in clique) <= 1)

# for clique in E_v:
#     for a in clique:
#         for b in clique:
#             if a >= b:
#                 continue
#             Chi[a, b] = solver.BoolVar('chi[%i,%i]' % (a, b))
    
#     s = solver.Sum(Chi[a, b] * 1 for a in clique for b in clique if b > a)
#     Sum.append(s)
    
#     for a in clique:
#         for b in clique:
#             if a >= b:
#                 continue
#             solver.Add(2 * Chi[a, b] <= Rho[a] + Rho[b])
    
# solver.Maximize(solver.Sum(Sum))
# RESULT = solver.Solve()
# print("Time = ", solver.WallTime(), " ms")
# print("result:", RESULT)
# print('\nTotal cost:', solver.Objective().Value())

# for idx in range(n):
#     print('node ' + str(idx) + ' -> ', Rho[idx].solution_value())

# print('\n===========================')
# for (a,b), v in Chi.items():
#     print(str((a, b)) + ' -> ', v.solution_value())